In [1]:
import copy
import math
import random
import numpy as np

In [2]:
datas = np.load('data/data.npy')
targets = np.load('data/target.npy')
datas.shape, targets.shape

((10, 20, 20), (10, 20, 20))

In [ ]:
from collections import deque
memory = deque(maxlen=1000)

def append_sample(state, mask, target, reward):
    memory.append((state, mask, target, reward))

In [ ]:
def get_furniture_list(data):
    furnitures = {}
    for i in range(1, 17):
        furnitures[i] = [[]]
        for j in range(20):
            for k in range(20):
                if data[j][k] == i:
                    furnitures[i][j].append(data[j][k])
            furnitures[i].append([])

    for i in range(1, 17):
        for j in range(20, -1, -1):
            if len(furnitures[i][j]) == 0:
                del(furnitures[i][j])
            
    for i in range(16, 0, -1):
        if len(furnitures[i]) == 0:
            del(furnitures[i])
    
    for key in furnitures.keys():
        furnitures[key] = np.array(furnitures[key])
    
    return furnitures